#### Catch22 features for all dataset

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.util import to_pandas
from gluonts.dataset.common import ListDataset
import pycatch22

In [ ]:
data = pd.DataFrame()
for i in ["electricity", "traffic", "m4_hourly", "m4_daily", "m4_weekly", "m4_monthly", "m4_quarterly", "solar-energy"]:
   dataset = get_dataset(i)
   train_ds = ListDataset(dataset.train, freq=dataset.metadata.freq)
   dff = pd.DataFrame()
   for ts in iter(train_ds):
      train_series = to_pandas(ts)
      df = pd.DataFrame(pycatch22.catch22_all(list(train_series)))
      df = df.T
      new_header = df.iloc[0] #grab the first row for the header
      df = df[1:] #take the data less the header row
      df.columns = new_header #set the header row as the df header
      
      dff = dff.append(df)
   dff = dff.reset_index(drop=True)
   data = data.append(pd.DataFrame(dff.mean(axis=0)).T.reset_index(drop=True))
   
data['dataset'] = ["electricity", "traffic", "m4_hourly", "m4_daily", "m4_weekly", "m4_monthly", "m4_quarterly", "solar-energy"]
data.to_csv('catch22.csv', index=False)

### Regression experiments

In [ ]:
import pandas as pd
# import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedShuffleSplit
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:

from sklearn.metrics import \
    r2_score, get_scorer
from sklearn.linear_model import \
    Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import \
    StandardScaler, PolynomialFeatures
from sklearn.model_selection import \
    KFold, RepeatedKFold, GridSearchCV, \
    cross_validate, train_test_split
from sklearn.linear_model import ElasticNet

In [ ]:
df = pd.read_csv("df.csv")
catch22 = pd.read_csv("catch22.csv")
data = df.merge(catch22, how='left', on="dataset")

In [ ]:
data.head()

In [ ]:
corr = data[data['model']=='transformer'].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
corr = data[data['model']=='Hopfield'].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
corr = data[data['model']=='switch'].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
corr = data[data['model']=='Informer'].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
corr = data[data['model']=='etsformer'].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# dataset = pd.get_dummies(data['dataset'])
# data = data.merge(dataset, how='outer',left_index=True, right_index=True)
data = data.drop(['dataset'], axis=1)
data.model = pd.Categorical(data.model)
data['code'] = data.model.cat.codes
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(data.drop(['model'], axis=1), data['code'])

seen data, seen model-- not possible, ideally would be same data point except that we perfrom straified split with respect to model type

### Test set: seen model, seen dataset(other model type has seen the dataset)

In [ ]:
for train_index, test_index in sss.split(data.drop(['model'], axis=1), data['code']):
    model_type =  pd.get_dummies(data['model'])
    data_temp = data.merge(model_type, how='outer',left_index=True, right_index=True)
    data_temp = data_temp.drop(['model'], axis=1)
    X = np.array(data_temp.drop(['error','code'], axis=1))[train_index]
    y = np.array(data_temp['error'])[train_index]
    reg = LinearRegression().fit(X, y)
    clf = linear_model.Lasso(alpha=0.01).fit(X, y)
    elastic_net = ElasticNet(alpha=0.04, l1_ratio=0)
    #lasso, elastic net,gridsearch on hyperparameter, more statistics, remove onehot encoding(dataset), add ett dataset 
    #4 quadrants of testing(unseen data, unseen model)
    #have some sparsity(lasso)


In [ ]:
temp=pd.DataFrame()
temp_test = pd.DataFrame()
for i in list(train_index):
    temp = temp.append(data.iloc[i,:])
for i in list(test_index):
    temp_test = temp_test.append(data.iloc[i,:])

In [ ]:
X.shape

In [ ]:
X_test = np.array(data_temp.drop(['error','code'], axis=1))[test_index]
y_test = np.array(data_temp['error'])[test_index]
y_pred = reg.predict(X_test)
y_pred_train = reg.predict(X)
print('test')
print('mse prediction:',mean_squared_error(y_test, y_pred))
print("R2 score prediction", r2_score(y_test, y_pred))
print("train")
print('mse prediction:',mean_squared_error(y, y_pred_train))
print("R2 score prediction", r2_score(y, y_pred_train))

In [ ]:
print('mse prediction:',mean_squared_error(y, clf.predict(X)))
print("R2 score prediction", r2_score(y, clf.predict(X)))
print('test')
print('mse prediction:',mean_squared_error(y_test, clf.predict(X_test)))
print("R2 score prediction", r2_score(y_test, clf.predict(X_test)))

In [ ]:
temp = pd.DataFrame()
temp['features'] = list(data_temp.drop(['error','code'], axis=1).columns)
temp['coefficient'] = reg.coef_
temp

In [ ]:
plt.scatter(y, y_pred_train)
plt.xlabel("true value")
plt.ylabel("predicted value")

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("true value")
plt.ylabel("predicted value")

### Test set: seen model, unseen dataset

In [ ]:
model_type =  pd.get_dummies(data['model'])
data_temp = data.merge(model_type, how='outer',left_index=True, right_index=True)
test_data = data_temp[(data_temp['dataset']=='m4_monthly') | (data_temp['dataset']=='solar-energy')].reset_index(drop=True).drop(['model', 'dataset'], axis=1)
train_data = data_temp[(data_temp['dataset']!='m4_monthly') & (data_temp['dataset']!='solar-energy')].reset_index(drop=True).drop(['model', 'dataset'], axis=1)
reg = LinearRegression().fit(train_data.drop(['error'], axis=1), train_data['error'])

In [ ]:
test_data.shape

In [ ]:
X_test = test_data.drop(['error'], axis=1)
y_test = test_data['error']

In [ ]:
y_pred = reg.predict(X_test)
y_pred_train = reg.predict(train_data.drop(['error'], axis=1))

In [ ]:
# print('mse prediction:',mean_squared_error(y, clf.predict(X)))
# print("R2 score prediction", r2_score(y, clf.predict(X)))

In [ ]:

print('MSE:',mean_squared_error(y_test, y_pred))
print("R2 score:", r2_score(y_test, y_pred))

In [ ]:
print('MSE:',mean_squared_error(train_data['error'], y_pred_train))
print("R2 score:", r2_score(train_data['error'], y_pred_train))

Coefficients

In [ ]:
temp = pd.DataFrame()
temp['features'] = list(train_data.drop(['error'], axis=1).columns)
temp['coefficient'] = reg.coef_
temp

In [ ]:
plt.rcParams["figure.figsize"] = (5,4)

In [ ]:
plt.scatter(train_data['error'], y_pred_train)
plt.xlabel("true value")
plt.ylabel("predicted value")

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("true value")
plt.ylabel("predicted value")

#### seen data, unseen model

In [ ]:
df = pd.read_csv("df.csv")
catch22 = pd.read_csv("catch22.csv")
data = df.merge(catch22, how='left', on="dataset")



In [ ]:
test_data = data[(data['model']=='linformer')|(data['model']=='Informer')].reset_index(drop=True).drop(['model', 'dataset'], axis=1)
train_data = data[(data['model']!='linformer')&(data['model']!='Informer')].reset_index(drop=True).drop(['model', 'dataset'], axis=1)
reg = LinearRegression().fit(train_data.drop(['error'], axis=1), train_data['error'])



In [ ]:
X_test = test_data.drop(['error'], axis=1)
y_test = test_data['error']
y_pred = reg.predict(X_test)
y_pred_train = reg.predict(train_data.drop(['error'], axis=1))

In [ ]:
train_data.shape

In [ ]:

print("Test data")
print('MSE:',mean_squared_error(y_test, y_pred))
print("R2 score:", r2_score(y_test, y_pred))
print("Train data")
print('MSE:',mean_squared_error(train_data['error'], y_pred_train))
print("R2 score:", r2_score(train_data['error'], y_pred_train))

In [ ]:
plt.scatter(train_data['error'], y_pred_train)
plt.xlabel("true value")
plt.ylabel("predicted value")

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("true value")
plt.ylabel("predicted value")

#### unseen data, unseen model

In [ ]:
df = pd.read_csv("df.csv")
catch22 = pd.read_csv("catch22.csv")
data = df.merge(catch22, how='left', on="dataset")
model_type =  pd.get_dummies(data['model'])
data_temp = data.merge(model_type, how='outer',left_index=True, right_index=True)
test_data = data_temp[((data_temp['model']=='linformer')|(data_temp['model']=='Informer') | (data_temp['model']=='nystorm')) & ((data_temp['dataset']=='m4_quarterly') | (data_temp['dataset']=='m4_weekly') | (data_temp['dataset']=='m4_daily'))].reset_index(drop=True).drop(['model', 'dataset'], axis=1)
train_data = data_temp[(data_temp['model']!='linformer')&(data_temp['model']!='Informer')&(data_temp['model']!='nystorm')&(data_temp['dataset']!='m4_quarterly')& (data_temp['dataset']!='m4_weekly') & (data_temp['dataset']!='m4_daily')].reset_index(drop=True).drop(['model', 'dataset'], axis=1)

train_x = train_data.drop(['error'],axis=1)
train_y = train_data['error']
test_x = test_data.drop(['error'], axis=1)
test_y = test_data['error']


reg = LinearRegression().fit(train_x, train_y)
y_pred = reg.predict(test_x)
y_pred_train = reg.predict(train_x)
print("Test data")
print('mse prediction:',mean_squared_error(test_y, y_pred))
print("R2 score prediction", r2_score(test_y, y_pred))
print("Train data")
print('mse prediction:',mean_squared_error(train_y, y_pred_train))
print("R2 score prediction", r2_score(train_y, y_pred_train))


In [ ]:
train_x.shape

In [ ]:
data_temp[((data_temp['model']=='linformer')|(data_temp['model']=='Informer') | (data_temp['model']=='nystorm')) & ((data_temp['dataset']=='m4_quarterly') | (data_temp['dataset']=='m4_weekly') | (data_temp['dataset']=='m4_daily'))].reset_index(drop=True)

In [ ]:
###lasso

In [ ]:
for train_index, test_index in sss.split(data.drop(['model'], axis=1), data['code']):
     model_type =  pd.get_dummies(data['model'])
     data_temp = data.merge(model_type, how='outer',left_index=True, right_index=True)
     data_temp = data_temp.drop(['model'], axis=1)
     X = np.array(data_temp.drop(['error','code'], axis=1))[train_index]
     y = np.array(data_temp['error'])[train_index]
     

     sc = StandardScaler()
     X_scaled = sc.fit_transform(X)
     # X_scaled = pd.DataFrame(data = X_scaled, columns = X.columns)
     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
     lasso_alphas = np.linspace(0, 0.2, 21)
     lasso = Lasso()
     grid = dict()
     grid['alpha'] = lasso_alphas
     gscv = GridSearchCV( lasso, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
     results = gscv.fit(X_scaled, y)
     print('MSE: %.5f' % results.best_score_)
     print('Config: %s' % results.best_params_)

In [ ]:

#elastic net


for train_index, test_index in sss.split(data.drop(['model'], axis=1), data['code']):
    model_type =  pd.get_dummies(data['model'])
    data_temp = data.merge(model_type, how='outer',left_index=True, right_index=True)
    data_temp = data_temp.drop(['model'], axis=1)
    X = np.array(data_temp.drop(['error','code'], axis=1))[train_index]
    y = np.array(data_temp['error'])[train_index]


    sc = StandardScaler()
    X_scaled = sc.fit_transform(X)
    # X_scaled = pd.DataFrame(data = X_scaled, columns = X.columns)
    param_grid = [
    {'alpha': np.linspace(0, 0.2, 21), 'l1_ratio': [0, 0.2, .5, .8, 1]},]
    elastic_net = ElasticNet()
    grid_search = GridSearchCV(elastic_net, param_grid, cv=5,
                            scoring='neg_mean_squared_error',
                            return_train_score=True)
    grid_search.fit(X, y)
    cvres = grid_search.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score), params)





In [ ]:
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))
        plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
        plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

In [ ]:
plot_learning_curves(lasso,X,y)
plt.show()

In [ ]:
plot_learning_curves(elastic_net,X,y)
plt.show()